In [24]:
from random import random
import generators as gen
from math import pi, log 

#TIEMPOS DE ARRIBO
def lamda_t(t):
    return 30 + 30*math.sin((2*np.pi*t)/24)
    
def Poisson_NH(T):
    NT = 0
    Eventos = []
    lamda = 7
    t = -log(1 - random()) / lamda
    while t <= T:
        V = random()
        if V < lamda_t(t) / lamda:
            NT += 1
            Eventos.append(t)
        t += -log(1 - random()) / lamda
    return NT, Eventos

def exp(lamda):
    u =  random()
    return -log(u)/lamda

In [29]:
def queueSystem(hours=48):
    n, arribos =  Poisson_NH(hours)
    servicios = [exp(40) for _ in arribos]
    esperas = []

    arriboAnt = 0
    for i in range(n):
        inicio = max(arribos[i], arriboAnt)
        espera = inicio - arriboAnt
        esperas.append(espera)
        arriboAnt = inicio + servicios[i]

    return arribos, servicios, esperas

queueSystem()

([0.04447281910386045,
  0.2587256011836938,
  0.5965612095525098,
  0.6704671969599465,
  0.8092468499490553,
  0.9112184256727474,
  0.9851857474834621,
  1.3063561124375709,
  1.5378189605768275,
  1.6277818566145268,
  1.6593439566711716,
  1.7204764776197934,
  1.837822062708365,
  1.9685305995300912,
  2.246358902298445,
  2.2676148454141734,
  2.2811355319013513,
  2.3934691728538313,
  2.5010075769563826,
  2.5927084965806695,
  2.8608517264962865,
  2.8668797079089954,
  3.0630232794182657,
  3.1158323247621746,
  3.120073522262436,
  3.1496785526773925,
  3.1565565220905687,
  3.2265815951497845,
  3.2354173072113324,
  3.3421254398468827,
  3.4045627478686145,
  3.408652747796347,
  3.4688400105988,
  3.911790953839343,
  4.155490398486733,
  4.314350173930325,
  4.50827254487236,
  4.535404788996895,
  4.608010786862558,
  4.783112000389254,
  5.115489097236942,
  5.385796521988021,
  5.519328662396515,
  5.769461051432355,
  6.035573616681632,
  6.271007127854523,
  6.3663